In [16]:
import pandas as pd
from pyECLAT import ECLAT

### I. Importing the dataset

In [9]:
snacks = pd.read_csv('snacks.csv')

snacks.head()

,Bier,Chips,Coca Cola,Eiscreme,Erdnussflips,Erdnüsse,Flips,Gummibärchen,Ich mag keine Snacks,Karotten,...,Pistazien,Popcorn (ohne Zusatz),Popcorn (salzig),Popcorn (süß),Salzstangen,Saure Gummibaecht,Schokolade,Snacktomaten,Softdrinks,Toffiffee
0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [12]:
snacks_eclat = pd.read_csv('snacks_eclat.csv')

snacks_eclat.head()

,0,1,2,3,4,5,6,7,8
0,Chips,Popcorn (süß),Erdnüsse,M&Ms,NaN,NaN,NaN,NaN,NaN
1,Nachos,M&Ms,Lecker Bierchen,NaN,NaN,NaN,NaN,NaN,NaN
2,Chips,Popcorn (süß),Nachos,M&Ms,NaN,NaN,NaN,NaN,NaN
3,Chips,Nachos,Schokolade,Erdnüsse,Kekse,M&Ms,Eiscreme,Bier,NaN
4,Chips,Nachos,Kekse,Salzstangen,NaN,NaN,NaN,NaN,NaN


In [10]:
items_total = snacks.astype(int).sum(axis=0)
items_total


Bier                       2
Chips                    123
Coca Cola                  1
Eiscreme                  31
Erdnussflips               1
Erdnüsse                  40
Flips                      1
Gummibärchen              47
Ich mag keine Snacks       1
Karotten                   1
Keine                      1
Kekse                     37
Kichererbsen               1
Knabberbox                 1
Käse-Cracker               1
Lecker Bierchen            1
M&Ms                      37
Nachos                    72
Obst                       1
Pistazien                  1
Popcorn (ohne Zusatz)      1
Popcorn (salzig)          20
Popcorn (süß)             55
Salzstangen                5
Saure Gummibaecht          1
Schokolade                51
Snacktomaten               1
Softdrinks                 1
Toffiffee                  1
dtype: int64

In [14]:
items_per_transaction = snacks.astype(int).sum(axis=1)
items_per_transaction

0      4
1      3
2      4
3      8
4      4
      ..
185    1
186    2
187    9
188    3
189    3
Length: 190, dtype: int64

In [23]:
# dataframe column names have to be numbers in order for the ECLAT algorithm to work

dataFrame = snacks_eclat.copy()
attrCount = len(dataFrame.count())
i = 0
for col in dataFrame.columns[:attrCount].tolist():
    dataFrame.rename(columns={col : i}, inplace=True)
    i += 1

eclat_instance = ECLAT(dataFrame, verbose=True)

100%|██████████| 29/29 [00:00<00:00, 4111.65it/s]


In [24]:
# the item shoud appear at least at 5% of transactions
min_support = 5/100

# start from transactions containing at least 2 items
min_combination = 2

# up to maximum items per transaction
max_combination = max(items_per_transaction)
rule_indices, rule_supports = eclat_instance.fit(min_support=min_support,
                                                 min_combination=min_combination,
                                                 max_combination=max_combination,
                                                 separator=' & ',
                                                 verbose=True)

Combination 2 by 2


45it [00:00, 189.13it/s]


Combination 3 by 3


120it [00:00, 252.92it/s]


Combination 4 by 4


...

120it [00:00, 209.81it/s]


Combination 8 by 8


45it [00:00, 210.49it/s]


Combination 9 by 9


10it [00:00, 190.21it/s]


In [25]:
result = pd.DataFrame(rule_supports.items(),columns=['Item', 'Support'])
result.sort_values(by=['Support'], ascending=False)

,Item,Support
6,Nachos & Chips,0.257895
21,Popcorn (süß) & Chips,0.189474
33,Chips & Gummibärchen,0.173684
16,Erdnüsse & Chips,0.168421
30,Schokolade & Chips,0.163158
12,M&Ms & Chips,0.126316
8,Nachos & Gummibärchen,0.121053
28,Kekse & Chips,0.121053
2,Nachos & Popcorn (süß),0.110526
20,Popcorn (süß) & Schokolade,0.110526
